<a href="https://colab.research.google.com/github/tomhyhan/noodles/blob/main/pasta_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import userdata
from google.colab import auth

auth.authenticate_user()
token = userdata.get('token')
username = userdata.get('username')
repo_name = "noodles"

In [3]:
!git clone https://github.com/tomhyhan/{repo_name}.git
%cd {repo_name}
!git pull

fatal: destination path 'noodles' already exists and is not an empty directory.
/content/noodles
Already up to date.


In [4]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import sys

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "noodles"

GOOGLE_DRIVE_PATH = os.path.join("..", "drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)


In [6]:
from model.utils import test_colab
test_colab()


Hello Colab! from Noodles ha


In [7]:
!pip install torchinfo
!pip install imagehash

In [8]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold


from config.config_manager import ConfigManager
from collections import Counter
from model.data_model import PastaData, create_train_transforms, create_test_transforms
from model.train import trainer, create_model
from model.utils import reset_seed
from model.data import CLASS_ENCODER, create_csv
from model.viz import class_imblance

In [9]:
config_manager = ConfigManager("./config/config.yml")

SEED = config_manager.config.seed

In [10]:
reset_seed(SEED)

In [11]:
image_path = os.path.join(GOOGLE_DRIVE_PATH, "images")
csv_file_path = os.path.join(GOOGLE_DRIVE_PATH, "pasta_data.csv")

# create_csv(image_path, csv_file_path)



In [12]:
data = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "./pasta_data.csv"))
image_paths, labels = data["img_path"], data["label"]

X, test_data, y, test_label = train_test_split(image_paths.values, labels.values, train_size=0.9, random_state=SEED, shuffle=True, stratify=labels)

In [13]:
k_fold = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
models = ['swin']

N=100
perm_indices = np.random.permutation(N)
X = X[perm_indices]
y = y[perm_indices]

for model_name in models:
    for k_id, (train_i, val_i) in enumerate(k_fold.split(X,y)):
        print(len(train_i), len(val_i))
        train_set = X[train_i]
        train_label_set = y[train_i]

        val_set = X[val_i]
        val_label_set = y[val_i]

        model_config = config_manager.config[model_name]
        lr = config_manager.config.lr
        num_epochs = config_manager.config.num_epochs
        num_classes = config_manager.config.num_classes
        batch_size = model_config.train_args.batch_size
        weight_decay = model_config.train_args.weight_decay
        log_interval = model_config.train_args.log_interval
        out_dir = model_config.train_args.output_dir

        device = 'cuda' if torch.cuda.is_available() else 'cpu'

        train_transform = create_train_transforms()
        test_transform = create_test_transforms()

        train_data = PastaData(train_set, train_label_set, transform_list=train_transform)
        val_data = PastaData(val_set, val_label_set, transform_list=test_transform)

        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, drop_last=True)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)

        model = create_model(model_name, num_classes)
        model.to(device)

        #  need to make dataframe from images
        loss_history, train_accuracy_history, \
        val_accuracy_history, best_accuracy, best_params = trainer(
            model,
            train_loader,
            val_loader,
            num_epochs=num_epochs,
            lr=lr,
            batch_size=batch_size,
            weight_decay=weight_decay,
            log_interval=log_interval,
            device=device
        )

        os.makedirs(os.path.join(GOOGLE_DRIVE_PATH, out_dir), exist_ok=True)
        torch.save(model.state_dict(), os.path.join(GOOGLE_DRIVE_PATH, out_dir, f"{model_name}_k_fold_{k_id}.pt"))
        print(f"best accuracy: {best_accuracy}")
        break
    break
        # avg_accuracy = torch.mean(fold_accuracies)
        # std_accuracy = torch.std(fold_accuracies)

        # general plan
        # 1. save best accuracies
        # 2. compare with different models
        # 3. train on full dataset with best performing model
        # 4. make inference


6291
<class 'numpy.ndarray'>
160 40


Epoch 1/1:   0%|          | 0/15 [00:00<?, ?it/s]

y 32
y 32
y 32
y 32
y 32
y 32
y 32
y 32
y 32
y 32
y 32
y 32
y 32
y 32
y 32
total 480
y 32
total 32
Learning Rate: [0.0001]
Epoch 1 Loss: 2.7398741404215494 Train Accuracy: 0.3854166666666667


ValueError: not enough values to unpack (expected 5, got 2)

In [15]:
# !git config pull.rebase false
# !git pull
# !git config --global user.email "tomhyhan@gmail.com"
# !git config --global user.name username
# !git add .
# !git status
# !git commit -m "updated trainer"
# !git push https://{token}@github.com/{username}/{repo_name}.git

Already up to date.
On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   config/config.yml
	modified:   model/data.py
	modified:   model/train.py

[main 6249cd6] updated trainer
 3 files changed, 11 insertions(+), 12 deletions(-)
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 2 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 741 bytes | 741.00 KiB/s, done.
Total 7 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/tomhyhan/noodles.git
   3e3e6b9..6249cd6  main -> main
